# Vectorization: an introduction to universal functions

In this notebook, you will learn about
- When and how to vectorize code
- Why this works: ufuncs
- Saving data for later use with `np.savez`
- Basic built-in NumPy tools for plotting and integrating with Matplotlib.
- Examples and applications

---

## 1. Broadcasting

The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

In [ ]:
import numpy as np

NumPy operations are usually done on pairs of arrays on an element-by-element basis. In the simplest case, the two arrays must have exactly the same shape, as in the following example:

In [ ]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b

NumPy’s broadcasting rule relaxes this constraint when the arrays’ shapes meet certain constraints. The simplest broadcasting example occurs when an array and a scalar value are combined in an operation:

In [ ]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b

We can think of the scalar `b` being stretched during the arithmetic operation into an array with the same shape as `a`. The new elements in `b`, as shown in the figure below, are simply copies of the original scalar.

!["A scalar is broadcast to match the shape of the 1-d array it is being multiplied to."](https://numpy.org/devdocs/_images/broadcasting_1.svg)

We will continue using the example of [Notebook 1](01_Intro.ipynb):

In [ ]:
import pandas as pd

quality_of_life = pd.read_csv('../data/quality_of_life_index.csv')
quality_index = np.array(quality_of_life['Quality of Life Index'])
quality_cost_pollution = np.array(quality_of_life[['Quality of Life Index', 'Cost of Living Index', 'Pollution Index']]) 

From the [Numbeo website](https://www.numbeo.com/quality-of-life/indices_explained.jsp), the formula for the Quality of Life Index involves a combination of a weighted sum of the other indices in our `quality_of_life` DataFrame. Let's say we wanted to renormalize every value in our `quality_cost_pollution` array by the `quality_of_life` index. Because of the broadcasting properties of NumPy arrays, this can be done by putting together a few of the things we've learned:

In [ ]:
renormalized_values = quality_cost_pollution/quality_index[:, np.newaxis]

In [ ]:
renormalized_values

## 2. Universal Functions

These operations are called [Universal Functions (ufunc)](https://numpy.org/doc/stable/reference/generated/numpy.ufunc.html), which are functions that operate on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features. That is, a ufunc is a “vectorized” wrapper for a function that takes a fixed number of specific inputs and produces a fixed number of specific outputs.

This is what makes mathematical operations work with ndarrays. We've seen a few arithmetic operations, but it's also possible to apply boolean operations, like `<`, `>` or `==` to ndarrays. NumPy has a number of other `ufunc` functions that operate on each element, such as

- sine and cosine: `np.sin` and `np.cos`
- exponential $e^x$: `np.exp(x)`
- logarithms $\log(x)$: `np.log(x)`

and others.

Going back to our example, we can compute

In [ ]:
np.log(quality_index)

## 3. Plotting

With the [Matplotlib](https://matplotlib.org) library, we can create plots and graphs directly from NumPy arrays. First, let's import the `pyplot` submodule of the matplotlib library:

In [ ]:
import matplotlib.pyplot as plt

In order to create a regular 2D plot, we need to have one sequence representing values in the x-axis, and another representing the values in the y-axis. In our case, let's say we want to create a plot with both the `quality_index` and `cost_index` arrays, choosing the indices of the arrays as the x-axis values. Without going into too much detail on the matplotlib syntax, we can do:

In [ ]:
fig, ax = plt.subplots()  # Creates the figure and the axis elements, where we will plot our arrays

ax.plot(np.arange(len(quality_index)), quality_index, label='Quality Index')

plt.legend()  # Creates legends with labels set in each line plot
plt.show()  # Plots the result   

Note that because the matplotlib library is expecting to deal with NumPy ndarrays, it is also possible to plot a 2D array directly; in this particular case, one line plot is create for each column of our 2D array:

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(quality_index)), quality_cost_pollution, label=['Quality Index', 'Cost of Living Index', 'Pollution Index'])

plt.legend()  # Note that here we can set one label per column in our 2D array directly as a list of labels.
plt.show()

## 4. Putting it all together

By inspection, we can see that the curve representing the Quality of Life Index that we have obtained has a familiar shape:

In [ ]:
plt.plot(quality_index) 

We could try to use universal functions to verify our hypothesis. Using the inverse of the [logistic function](https://en.wikipedia.org/wiki/Logistic_function), we can identify a trend in the data we have. First, define a function which computes the inverse of the logistic function for all allowed values of $x$: 

In [ ]:
def inverse_logistic(x):
    return (1/0.05)*np.log(10/x - 1)+100

Note that we have defined this function as we would for a scalar variable `x` - we only made sure to use `np.log` to compute the logarithm. Now, we can build the domain where we will compute this function:

In [ ]:
domain = np.linspace(0.1, 9.9)

Finally, let's plot this curve:

In [ ]:
plt.plot(inverse_logistic(domain))

---

#### Self-assessment 1

---

## 5. Another problem to solve 

Consider the motion of a ball thrown from one person to another. The path of the ball is defined here as

- $x(t) = x_0 + v_x t$ the forward distance of the ball
- $y(t) = y_0 + v_y t - \frac{g}{2}t^2$ the height of the ball

where

- $x_0 = 0~m$ is the initial distance travelled
- $v_x$ is the initial forward speed of the ball
- $y_0$ is the initial height of the ball
- $v_y$ is the initial upward speed of the ball
- $g = 9.81~\frac{m}{s^2}$ is the acceleration due to gravity

Let's use NumPy to find all the locations where a 175-cm person can stand to catch a ball thrown from $x_0=0~m$ and $y_0=2~m$ high at $v_x = 3.5~m$ and $v_y = 4.5~m$. 

You will do this in five steps:

1. define your constants: $x_0,~v_x,~y_0,~v_y,~g$
2. define your independent variable, time, as a NumPy array
3. calculate the positions using time and your constants
4. plot the path of the ball
5. find the x-locations where $y(t)<0.175~m$

### 5.1. Define your constants

First, define the variables that are constant in the functions, $x(t)$ and $y(t)$:

- $x_0$ as `x0`
- $y_0$ as `y0`
- $v_x$ as `vx`
- $v_y$ as `vy`
- $g$ as `g`

In [ ]:
x0 = 0
y0 = 2

vx = 3.5
vy = 4.5
g = 9.81

### 5.2. Define your independent variable, time, as a NumPy array

Now, you will use a built-in NumPy function, [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) to create the independent variable $t$ as `time`. The function `np.linspace` uses three arguments to define an array, as such

```python
time = np.linspace(start, end, number_of_steps)
```

- the `start` creates the first value in the array `time`
- the `end` creates the last value in the array `time`
- the `number_of_steps` defines how many steps to take between `start` and `end`

In [ ]:
time = np.linspace(0, 1, 21)

**Note** `np.linspace` is very similar to `np.arange`, but you should be aware that `np.arange` may give unexpected results due to loss of precision. Check [the `np.arange` docstring](https://numpy.org/devdocs/reference/generated/numpy.arange.html) for more details.

Consider the output from the array, `time`. Here you `print` some descriptions of how `np.linspace` defined your array. 

In [ ]:
print('Your independent variable is time')
print('time starts at {} s and ends at {} s'.format(time[0], time[-1]))
print()
print('it has {} time steps and each step is {} s'.format(len(time), time[1] - time[0]))

### 5.3. Calculate the positions using time and your constants

Now, that you have a NumPy array, you can plug it directly into equations to create new arrays. Next you define 

- `x` as $x(t) = x_0 + v_x t$ the forward distance of the ball
- `y` as $y(t) = y_0 + v_y t - \frac{g}{2}t^2$ the height of the ball

NumPy arrays make defining these functions straightforward. In the array `x`, each value of time [0, 0.05, ..., 1] s was multiplied by the initial speed, `vx`, then added to the initial position, `x0`. No need to create a `for`-loop and define each value of `x`. 

In [ ]:
x = x0 + vx*time
y = y0 + vy*time -g/2*time**2

### 5.4. Plot the path of the ball

Now, you have defined three arrays, `t`, `x`, and `y`. Each of these arrays is the same shape `(21, )`. 

In [ ]:
print(np.shape(time))
print(np.shape(x))
print(np.shape(y))

Because they are all the same shape, you can use [Matplotlib](https://matplotlib.org/stable/index.html) to see how the ball will travel through the air. Here, you use `plt.plot` to plot $y(t)$ vs $x(t)$. 

In [ ]:
plt.plot(x, y, 'o')
plt.xlabel('forward distance (m)')
plt.ylabel('height (m)')

By plotting the path of the ball, notice how the height is over 2 meters for most of the time in the air. Somewhere past $x(t) = 3~m$ the ball is getting back to its original height of $2~m$. Next, you can find what those positions and times are. 

### 5.5. Find the x-locations where $y(t)<0.175~m$

Now, you want to find the locations to stand _and even when to catch the ball_. You can use another `ufunc` function that operates on every element in the variable `y`. Use the `<` operator to check when $y(t) < 1.75~m$

In [ ]:
y < 1.75

The result is a list of `True` and `False` statements. 
- If the value of `y` is more than $1.75~m$, then the result is `False` 
- If the value of `y` is less than $1.75~m$, then the result is `True`

Now, you can use this list of `True`/`False` statements to just look at the times and positions of the ball when the statement is `True`, using advanced indexing:

In [ ]:
time[y < 1.75]

In [ ]:
x[y < 1.75]

In [ ]:
y[y < 1.75]

In [ ]:
plt.plot(x, y, 'o')
plt.text(x[y < 1.75], y[y < 1.75], '<- Stand here')
plt.xlabel('forward distance (m)')
plt.ylabel('height (m)')

The result is that at $t = 1~s$, the ball has travelled $x(t=1) = 3.5~m$ and will be $y(t=1) = 1.595~m$ high. This is a great place to stand to catch the ball, just be ready!

---

## Read more
- [NumPy functions and methods overview](https://numpy.org/devdocs/user/quickstart.html#functions-and-methods-overview)
- [NumPy Quickstart guide](https://numpy.org/devdocs/user/quickstart.html)
- [NumPy for absolute beginners](https://numpy.org/devdocs/user/absolute_beginners.html)
- [Broadcasting](https://numpy.org/devdocs/user/basics.broadcasting.html)

## Next

Go to [Notebook 4: Submodules](04_Submodules.ipynb).